In [2]:
from requests import get

def get_movies_from_tastedive(name):
    '''Return 5 movies associated with the name from the database
    of TasteDive.
    
    Args:
        name: a string, name of a movie or an artist
        
    Returns:
        a dictionary in the form of
        {'Similar': {'Info': [{'Name': , 'Type': }], 'Results': [{'Name': , 'Type': },...]}}'''
    baseurl = 'https://tastedive.com/api/similar'
    param_dic = {'limit': 5, 'q': name, 'type': 'movies'}
    return get(baseurl, params=param_dic).json()

In [3]:
def extract_movie_titles(movie_dic):
    '''Extract the list of movie title from the dictionary returned
    by TasteDive.

    Args:
        movie_dic: a dictionary returned from tastedive in the format of
                   {'Similar': {'Info': [{'Name': , 'Type': }], 'Results': [{'Name': , 'Type': },...]}}
    Returns:
        a list of movie titles'''
    return [title['Name'] for title in movie_dic['Similar']['Results']]

print(extract_movie_titles(get_movies_from_tastedive('Black Panther')))

['Creed', 'Ready Player One', 'Ant-Man And The Wasp', 'Deadpool 2', 'Thor: Ragnarok']


In [4]:
def get_related_titles(movie_list):
    '''Take a list of movie tiles as inputs, find 5 related movie titles from
    TasteDive, combine them into a single list with no repetition
    
    Args:
        movie_list: a list of movie titles
        
    Returns:
        a list of related movie titles'''
    assoc_set = set()
    for title in movie_list:
        assoc_set.update(extract_movie_titles(get_movies_from_tastedive(title)))
    return list(assoc_set)

print(get_related_titles(["Black Panther", "Captain Marvel"]))

['Ant-Man And The Wasp', 'Avengers: Endgame', 'Aquaman', 'Creed', 'Shazam!', 'Spider-Man: Far From Home', 'Thor: Ragnarok', 'Ready Player One', 'Deadpool 2']


In [5]:
def get_movie_data(movie_title):
    '''Get movie data from OMDB
    Args:
        movie_title: a string with the movie title
        
    Returns:
        a dictionary containing the information of the movie'''
    baseurl = 'http://www.omdbapi.com/'
    param_dic = {'apikey': '3dceaff', 't': movie_title, 'r': 'json'}
    return get(baseurl, params=param_dic).json()
print(get_movie_data('Black Panther'))

{'Title': 'Black Panther', 'Year': '2018', 'Rated': 'PG-13', 'Released': '16 Feb 2018', 'Runtime': '134 min', 'Genre': 'Action, Adventure, Sci-Fi', 'Director': 'Ryan Coogler', 'Writer': 'Ryan Coogler, Joe Robert Cole, Stan Lee', 'Actors': "Chadwick Boseman, Michael B. Jordan, Lupita Nyong'o", 'Plot': "T'Challa, heir to the hidden but advanced kingdom of Wakanda, must step forward to lead his people into a new future and must confront a challenger from his country's past.", 'Language': 'English, Swahili, Nama, Xhosa, Korean', 'Country': 'United States', 'Awards': 'Won 3 Oscars. 116 wins & 281 nominations total', 'Poster': 'https://m.media-amazon.com/images/M/MV5BMTg1MTY2MjYzNV5BMl5BanBnXkFtZTgwMTc4NTMwNDI@._V1_SX300.jpg', 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.3/10'}, {'Source': 'Rotten Tomatoes', 'Value': '96%'}, {'Source': 'Metacritic', 'Value': '88/100'}], 'Metascore': '88', 'imdbRating': '7.3', 'imdbVotes': '734,689', 'imdbID': 'tt1825683', 'Type': 'movie', 'D

In [6]:
def get_movie_rating(omdb_dic):
    '''Take an omdb dictionary of a movie title, extract the Rotten
    Tomatoes score
    
    Args:
        omdb_dic: a dictionary returned from omdbapi.com, containg
        the information of a movie
        
    Returns:
        an integer, the Rotten Tomatoes score of the movie. If it does not have
        one, return 0'''
    try:
        for score in omdb_dic['Ratings']:
            if score['Source'] == 'Rotten Tomatoes':
                score_list = score['Value'].split('%')
                return int(score_list[0])
    except KeyError:
        return 0
    return 0

print(get_movie_rating(get_movie_data('Black Panther')))

96


In [7]:
def get_sorted_recommendations(movie_list):
    '''Take a list of movie titles as input, return a sorted list of related movie
    titles (ranked by Rotten Tomatoes score). The code is not incompatible with
    Python 3.5 or lower due to the reimplementation of Python dictionary from
    Python 3.6 onwards.
    
    Args:
        movie_list(list): the list of movie titles
        
    Returns:
        list: The sorted recommended list
    
    '''
    rec_list = get_related_titles(movie_list)
    rec_dic = {}
    for item in rec_list:
        rec_dic[item] = get_movie_rating(get_movie_data(item))
    sorted_rec_dic = sorted(rec_dic, reverse=True, key=lambda k: (rec_dic[k], k))
    return list(sorted_rec_dic)

print(get_sorted_recommendations(["Bridesmaids", "Sherlock Holmes"]))

['The Railway Children', 'Pitch Perfect', 'Crazy, Stupid, Love.', 'The Heat', 'Franklyn', 'Tag', 'Unaccompanied Minors', 'Bride Wars', 'Valley Girl', 'Bullet Boy']
